# CCL 樹脂配方 - 探索性數據分析 (EDA)

本 Notebook 分析模擬數據的分布、相關性，並視覺化配方參數與物理性質的關係。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 設定中文字體
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 設定繪圖風格
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# 載入數據
df = pd.read_csv('../data/ccl_resin_simulation.csv')
print(f'數據筆數: {len(df)}')
print(f'欄位數: {len(df.columns)}')
df.head()

## 1. 數據概覽

In [ ]:
# 基本統計
df.describe().round(4)

In [ ]:
# 定義輸入和輸出變數
input_cols = ['Hardener_Eq_Ratio', 'Filler_Vol_Pct', 'FR_Wt_Pct', 
              'Toughener_Wt_Pct', 'Wash_Cycles', 'Residual_Cl_ppm']
output_cols = ['Dk_10GHz', 'Df_10GHz', 'Peel_Strength_N_mm', 'Tg_C', 'CTE_ppm']

print('輸入變數:', input_cols)
print('輸出變數:', output_cols)

## 2. 輸入變數分布

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

for i, col in enumerate(input_cols):
    axes[i].hist(df[col], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')
    axes[i].axvline(df[col].mean(), color='red', linestyle='--', label=f'Mean: {df[col].mean():.2f}')
    axes[i].legend()

plt.suptitle('輸入變數分布 (配方參數)', fontsize=14)
plt.tight_layout()
plt.show()

## 3. 輸出變數分布

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

colors = ['#e74c3c', '#e67e22', '#2ecc71', '#3498db', '#9b59b6']
for i, col in enumerate(output_cols):
    axes[i].hist(df[col], bins=30, edgecolor='black', alpha=0.7, color=colors[i])
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')
    axes[i].axvline(df[col].mean(), color='black', linestyle='--', label=f'Mean: {df[col].mean():.4f}')
    axes[i].legend()

axes[5].axis('off')  # 隱藏多餘的子圖
plt.suptitle('輸出變數分布 (物理性質)', fontsize=14)
plt.tight_layout()
plt.show()

## 4. 相關性分析

In [ ]:
# 計算相關係數矩陣
corr_matrix = df[input_cols + output_cols].corr()

# 繪製熱力圖
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            mask=mask, square=True, linewidths=0.5)
plt.title('相關係數矩陣', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# 輸入 vs 輸出 相關性
corr_input_output = df[input_cols].corrwith(df[output_cols].T).unstack()
corr_io = df[input_cols + output_cols].corr().loc[input_cols, output_cols]

plt.figure(figsize=(10, 6))
sns.heatmap(corr_io, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            linewidths=0.5, cbar_kws={'label': 'Correlation'})
plt.title('配方參數 vs 物理性質 相關性', fontsize=14)
plt.xlabel('物理性質 (輸出)')
plt.ylabel('配方參數 (輸入)')
plt.tight_layout()
plt.show()

## 5. 關鍵關係視覺化

In [ ]:
# 硬化劑當量比 vs Dk, Df
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].scatter(df['Hardener_Eq_Ratio'], df['Dk_10GHz'], alpha=0.5, c='steelblue')
axes[0].set_xlabel('Hardener Eq Ratio')
axes[0].set_ylabel('Dk @10GHz')
axes[0].set_title('硬化劑當量比 vs 介電常數')

axes[1].scatter(df['Hardener_Eq_Ratio'], df['Df_10GHz'], alpha=0.5, c='coral')
axes[1].set_xlabel('Hardener Eq Ratio')
axes[1].set_ylabel('Df @10GHz')
axes[1].set_title('硬化劑當量比 vs 介電損耗')

plt.tight_layout()
plt.show()

In [ ]:
# 填料 vs CTE, Peel Strength
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].scatter(df['Filler_Vol_Pct'], df['CTE_ppm'], alpha=0.5, c='purple')
axes[0].set_xlabel('Filler Vol %')
axes[0].set_ylabel('CTE (ppm)')
axes[0].set_title('填料體積% vs 熱膨脹係數')

axes[1].scatter(df['Filler_Vol_Pct'], df['Peel_Strength_N_mm'], alpha=0.5, c='green')
axes[1].set_xlabel('Filler Vol %')
axes[1].set_ylabel('Peel Strength (N/mm)')
axes[1].set_title('填料體積% vs 剝離強度')

plt.tight_layout()
plt.show()

In [ ]:
# 增韌劑 vs Peel Strength, Tg
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].scatter(df['Toughener_Wt_Pct'], df['Peel_Strength_N_mm'], alpha=0.5, c='green')
axes[0].set_xlabel('Toughener Wt %')
axes[0].set_ylabel('Peel Strength (N/mm)')
axes[0].set_title('增韌劑 vs 剝離強度 (正相關)')

axes[1].scatter(df['Toughener_Wt_Pct'], df['Tg_C'], alpha=0.5, c='blue')
axes[1].set_xlabel('Toughener Wt %')
axes[1].set_ylabel('Tg (°C)')
axes[1].set_title('增韌劑 vs 玻璃轉移溫度 (負相關)')

plt.tight_layout()
plt.show()

## 6. Trade-off 分析

In [ ]:
# Dk vs Peel Strength (核心矛盾)
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df['Dk_10GHz'], df['Peel_Strength_N_mm'], 
                      c=df['Hardener_Eq_Ratio'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Hardener Eq Ratio')
plt.xlabel('Dk @10GHz (越低越好)')
plt.ylabel('Peel Strength (N/mm) (越高越好)')
plt.title('核心矛盾: 低 Dk vs 高 Peel Strength')

# 標記理想區域
plt.axvline(3.5, color='red', linestyle='--', alpha=0.5, label='Dk < 3.5 目標')
plt.axhline(0.7, color='green', linestyle='--', alpha=0.5, label='Peel > 0.7 目標')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# CTE vs Peel Strength
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df['CTE_ppm'], df['Peel_Strength_N_mm'], 
                      c=df['Filler_Vol_Pct'], cmap='plasma', alpha=0.6)
plt.colorbar(scatter, label='Filler Vol %')
plt.xlabel('CTE (ppm) (越低越好)')
plt.ylabel('Peel Strength (N/mm) (越高越好)')
plt.title('矛盾: 低 CTE vs 高 Peel Strength')

plt.axvline(35, color='red', linestyle='--', alpha=0.5, label='CTE < 35 目標')
plt.axhline(0.7, color='green', linestyle='--', alpha=0.5, label='Peel > 0.7 目標')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Peel Strength vs Tg (增韌劑的代價)
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df['Peel_Strength_N_mm'], df['Tg_C'], 
                      c=df['Toughener_Wt_Pct'], cmap='coolwarm', alpha=0.6)
plt.colorbar(scatter, label='Toughener Wt %')
plt.xlabel('Peel Strength (N/mm)')
plt.ylabel('Tg (°C)')
plt.title('矛盾: 高 Peel Strength vs 高 Tg')

plt.axvline(0.7, color='green', linestyle='--', alpha=0.5, label='Peel > 0.7 目標')
plt.axhline(160, color='blue', linestyle='--', alpha=0.5, label='Tg > 160 目標')
plt.legend()
plt.tight_layout()
plt.show()

## 7. 符合目標規格的配方統計

In [ ]:
# 定義目標規格
target_specs = {
    'Dk_10GHz': ('<=', 3.5),
    'Df_10GHz': ('<=', 0.010),
    'Peel_Strength_N_mm': ('>=', 0.7),
    'Tg_C': ('>=', 160),
    'CTE_ppm': ('<=', 35)
}

# 計算各項符合率
print('各項規格符合率:')
print('-' * 40)
for col, (op, val) in target_specs.items():
    if op == '<=':
        pass_rate = (df[col] <= val).mean() * 100
    else:
        pass_rate = (df[col] >= val).mean() * 100
    print(f'{col:25s} {op} {val:8.4f}: {pass_rate:5.1f}%')

# 全部符合
all_pass = (
    (df['Dk_10GHz'] <= 3.5) &
    (df['Df_10GHz'] <= 0.010) &
    (df['Peel_Strength_N_mm'] >= 0.7) &
    (df['Tg_C'] >= 160) &
    (df['CTE_ppm'] <= 35)
)
print('-' * 40)
print(f'全部符合: {all_pass.sum()} 筆 ({all_pass.mean()*100:.1f}%)')

In [ ]:
# 顯示符合所有規格的配方
if all_pass.sum() > 0:
    print('符合所有規格的配方:')
    display(df[all_pass].head(10))
else:
    print('沒有配方能同時符合所有規格 - 這正是需要 AI 優化的原因！')

## 8. EDA 結論

### 關鍵發現

1. **硬化劑當量比** 是控制 Dk/Df 的主要因素
   - 當量比 ↑ → Dk ↓, Df ↓ (好)
   - 但同時 Peel Strength ↓ (壞)

2. **填料體積%** 是控制 CTE 的主要因素
   - 填料 ↑ → CTE ↓ (好)
   - 但同時 Peel Strength ↓ (壞)

3. **增韌劑** 可以提升 Peel Strength
   - 增韌劑 ↑ → Peel ↑ (好)
   - 但同時 Tg ↓, Df ↑ (壞)

4. **核心矛盾**：很難同時達到低 Dk/Df + 高 Peel + 高 Tg + 低 CTE

### 下一步

建立機器學習模型，學習這些複雜的非線性關係，並用於：
1. 預測新配方的性質
2. 反向搜尋最佳配方